In [11]:
import requests, re,os
import pandas as pd

class TypeBase():
    def __init__(self,dictonary):
        for key in dictonary :
            target = self._converter(dictonary[key])
            setattr(self, key, target)
    def _converter(self,target):
        if type(target) is str:
            if re.match("^\d+?\.\d+?$", target):
                target = float(target)
            elif re.match("^\d+$", target):
                target = int(target)
        return target
    
class Instrument(TypeBase):
    def __init__(self,dictonary):
        super().__init__(dictonary)
        
class Host():
    def __init__(self,rest,stream):
        self.rest = rest 
        self.stream = stream 
        for name in ['rest','stream']:
            setattr(self, name, getattr(self, name) + '/v3/')
            
class Request():
    def __init__(self,key,type):
        if type is 'sandbox':
            self.host = Host('http://api-sandbox.oanda.com','http://stream-sandbox.oanda.com' )
        elif type is 'practice':
            self.host = Host('https://api-fxpractice.oanda.com','https://stream-fxpractice.oanda.com' )
        elif type is 'real':
            self.host = Host('https://api-fxtrade.oanda.com','https://stream-fxtrade.oanda.com' )
        else :
            raise Exception('unknown type')
        self.headers = {
            "Authorization" : 'Bearer ' + key,
            "X-Accept-Datetime-Format" : 'UNIX'
        }
    def get(self,url,params={}):
        response = requests.get(self.host.rest+url,headers=self.headers,params=params)
        json = response.json()
        err = None
        if 'errorMessage' in json:
            err = json['errorMessage']
            return err,None
        return None,json

class API():
    def __init__(self,key,type,id=None):
        self.request = Request(key,type)
        if id:
            self.account_id = id
            return;
        err,accounts = self.accounts()
        if err:
            raise Exception(err)
        using = None
        print('============== ACCOUNTS =================')
        for account in accounts:
            if using : 
                print(account['id'])
            else:
                using = account['id']
                print(account['id'] + ' << using this account.')
        self.account_id = using
    def _get(self,url,name,clas=None,params={}):
        err, data = self.request.get(url,params)
        if data and name in data:
            data = data[name]
            if clas :
                instanced = []
                for single in data:
                    instanced.append(clas(single))
                data = instanced
        return err , data
    def accounts(self):
        return self._get('accounts','accounts')
    def instruments(self,account_id=None):
        if account_id is None:
            account_id = self.account_id
        return self._get(
            'accounts/' + account_id + '/instruments',
            'instruments',
            Instrument)
    def candles(self,params):
        instrument = params['instrument']
        params.pop('instrument',None)
        if 'count' not in params:
            params['count'] = 5000
        err, candles = self._get(
            'instruments/'+instrument+'/candles',
            'candles',
            params=params)
        if candles is None:
            return err,candles
        if len(candles) == 0:
            return 'no more data',None
        temp = []
        for candle in candles:
            if 'mid' in candle:
                for key, value in {'o':'open','h':'high','l':'low','c':'close'}.items():
                    candle[value] = candle['mid'][key]
                candle.pop('mid',None)
                candle.pop('complete',None)
                temp.append(candle)
        candles = pd.DataFrame(temp)
        candles.time = pd.to_datetime(candles.time)
        #candles.time = candles.time.dt.tz_localize('UTC')
        candles = candles.set_index('time')
        candles = candles.reindex_axis(['open','high','low','close','volume'], axis=1)
        for name in ['open','high','low','close']:
            candles[name] = candles[name].astype('float')
        return err, candles
    
class OANDA():
    def __init__(self, key= None,type='practice',id=None):
        if key is None:
            raise Exception('need api key')
        self.api = API(key,type,id)

In [12]:
import configparser
# API接続のコンフィグを読み込み
config = configparser.ConfigParser()

#config.read('../Desktop/Oanda_API/config/config_v1.txt')
config.read('C:/Users/goomo/Desktop/Oanda_API/config//config_v1.txt')
account_id = config['oanda']['account_id']
api_key = config['oanda']['api_key']

In [14]:
oanda = OANDA(key=api_key, type='practice', id=account_id) 

In [16]:
def getHistorical(instrument,granularity,start):
    before = None;
    count = 1
    while(True):
        params = {
            'instrument' : instrument,
            'granularity' : granularity
        };
        if before is not None:
            params['to'] = before.iloc[0].name.timestamp()
        err,candles = oanda.api.candles(params)
        if err is not None:
            print(err)
            break;
        count += 1
        print("getting " + str(count), end="\r")
        if before is None:
            before = candles
        else:
            before = candles.append(before,verify_integrity=True)
    directory = "./data/"+instrument+"/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    before.to_csv(directory + granularity + ".csv", date_format='%Y-%m-%d %H:%M:%S')

In [19]:
import time

#instruments = ['USD_JPY', 'GBP_USD', 'EUR_USD']
instruments = ['JP225_USD']
granularities = ['M1']
start = ['2005']
for instrument in instruments:
    for granularity in granularities:
        print('start',instrument,granularity)
        start = time.time()
        getHistorical(instrument,granularity,start)
        print('end',instrument,granularity,time.time() - start)

start JP225_USD M1


C:\Users\goomo\Anaconda3\envs\gpu\lib\site-packages\ipykernel_launcher.py:113: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


no more data
end JP225_USD M1 803.533980846405
